# Account Routes

In [1]:
#| default_exp routes.account

In [2]:
#| exporti
from typing import Union
from enum import Enum
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

In [3]:
#| export
async def get_accounts(auth: dmda.DomoAuth,
                       debug_api: bool = False, 
                       session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None) -> rgd.ResponseGetData:
    """retrieve a list of all the accounts the user has read access to.  Note users with "Manage all accounts" will retrieve all account objects"""
    
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    return await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

#### sample implementation of get_accounts

In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)


res = await get_accounts(auth = token_auth)
pd.DataFrame(res.response)

,id,userId,name,displayName,type,valid,dataProviderType,credentialsType,createdAt,createdBy,modifiedAt,modifiedBy,configurations,accountId,accountTemplateId,accountTemplateAuthorizationId
0,1,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,dataset-copy,fields,1589100087000,1893952720,1589100087000,1893952720,{},1,None,None
1,5,1893952720,Domo Governance Datasets Third Party Account,test_rename,data,True,domo-governance-d14c2fef-49a8-4898-8ddd-f64998...,fields,1616777681000,1893952720,1689698195000,1893952720,{},5,None,None
2,27,1893952720,DataSet Copy Account,DataSet Copy Account,data,True,domo-csv,fields,1619083568000,1893952720,1619083568000,1893952720,{},27,None,None
3,45,1893952720,onyxReporting@gmail.com,test-goolesheets,data,True,google-spreadsheets,oauth,1664924354000,1893952720,1682350828000,1893952720,{},45,None,None
4,70,1893952720,jaemyong.wilson@sony.com,jaemyong.wilson@sony.com,data,True,google-spreadsheets,oauth,1682511651000,1893952720,1682511651000,1893952720,{},70,None,None
5,71,1893952720,domo_creds,domo_creds,data,True,abstract-credential-store,fields,1684447092000,1893952720,1684447092000,1893952720,{},71,None,None


In [5]:
#| export
class GetAccount_NoMatch(de.DomoError):
    def __init__(self, account_id, domo_instance, status, function_name = 'get_account_from_id'):

        message = f"account_id {account_id} not found"
        
        super().__init__(message = message, status = status, function_name = function_name , domo_instance = domo_instance)
    
async def get_account_from_id(auth: dmda.DomoAuth, account_id: int,
                              debug_api: bool = False, session: httpx.AsyncClient = None) -> rgd.ResponseGetData:
    """retrieves metadata about an account"""

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session, 
        timeout = 20 # occasionally this API has a long response time
    )

    if not res.is_success and (res.response == 'Forbidden' or res.response == 'Not Found'):
        raise GetAccount_NoMatch(
            account_id=account_id, domo_instance=auth.domo_instance, status=res.status)
    
    return res




#### sample implementation of get_account_from_id

In [6]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
try:
    res = await get_account_from_id(auth=token_auth, account_id = 45)
    print(res.response)
except GetAccount_NoMatch as e:
    print(e)


{'id': 45, 'userId': 1893952720, 'name': 'onyxReporting@gmail.com', 'displayName': 'test-goolesheets', 'type': 'data', 'valid': True, 'dataProviderType': 'google-spreadsheets', 'credentialsType': 'oauth', 'createdAt': 1664924354000, 'createdBy': 1893952720, 'modifiedAt': 1682350828000, 'modifiedBy': 1893952720, 'configurations': {}, 'accountId': 45, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None}


# Account Config

In [7]:
#| export
class AccountConfig_InvalidDataProvider(de.DomoError):
    def __init__(self, account_id:str, data_provider_type:str, status, domo_instance: str, function_name = 'route.get_account_config'):
        message = f"account {account_id} with data_provider_type '{data_provider_type}' does not return a config"

        super().__init__( message = message, status = status, function_name = function_name, domo_instance = domo_instance)

async def get_account_config(auth: dmda.DomoAuth,
                             account_id: int,
                             data_provider_type: str ,
                             debug_api: bool = False, 
                             session: Union[httpx.AsyncClient, httpx.AsyncClient, None] = None) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}?unmask=true"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if res.response == {} and res.is_success:
        raise AccountConfig_InvalidDataProvider(account_id= account_id, data_provider_type= data_provider_type, domo_instance=auth.domo_instance, status = res.status)
    
    return res

#### sample implementation of get_account_config

In [8]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    res = await get_account_config(auth=token_auth, account_id=5, data_provider_type='dataset-copy')
    res.response
except Exception as e:
    print(e)

route.get_account_config: Status 200 - account 5 with data_provider_type 'dataset-copy' does not return a config at domo-community


In [9]:
#| export
async def update_account_config(auth: dmda.DomoAuth,
                                account_id: int,
                                config_body: dict,
                                data_provider_type: str,
                                debug_api: bool = False, 
                                session: httpx.AsyncClient = None) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/providers/{data_provider_type}/account/{account_id}"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=config_body,
        debug_api=debug_api,
        session=session
    )

In [10]:
#| export
async def update_account_name(auth: dmda.DomoAuth,
                              account_id: int,
                              account_name: str,
                              debug_api: bool = False, 
                              session: httpx.AsyncClient = None) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/name"
    
    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=account_name,
        content_type = "text/plain",
        debug_api=debug_api,
        session=session
    )

In [11]:
#| export
async def create_account(auth:dmda.DomoAuth, config_body:dict,
                         debug_api: bool = False, session: httpx.AsyncClient = None) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body = config_body,
        debug_api=debug_api,
        session=session
    )

In [12]:
#| export
async def delete_account(auth:dmda.DomoAuth,
                         account_id: str,
                         debug_api: bool = False, 
                         session: httpx.AsyncClient = None) -> rgd.ResponseGetData:
    
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}"

    if debug_api:
        print(url)

    return await gd.get_data(
        auth=auth,
        url=url,
        method='DELETE',
        debug_api=debug_api,
        session=session
    )

# Account Sharing

In [13]:
# | export

class ShareAccount():
    pass


class ShareAccount_V1_AccessLevel(ShareAccount, Enum):
    CAN_VIEW = 'READ'


class ShareAccount_V2_AccessLevel(ShareAccount, Enum):
    CAN_VIEW = 'CAN_VIEW'
    CAN_EDIT = 'CAN_EDIT'
    CAN_SHARE = 'CAN_SHARE'


def generate_share_account_payload_v1(
    access_level: ShareAccount,
    user_id: int = None,
    group_id: int = None
):
    if user_id:
        return {"type": "USER", "id": int(user_id), "permissions": [access_level.value]}
    if group_id:
        return {"type": "GROUP", "id": int(group_id), "permissions": [access_level.value]}


def generate_share_account_payload_v2(
    access_level: ShareAccount,
    user_id: int = None,
    group_id: int = None
):

    if user_id:
        return {"type": "USER", "id": int(user_id), "accessLevel": access_level.value}

    if group_id:
        return {"type": "GROUP", "id": int(group_id), "accessLevel": access_level.value}


In [14]:
print(generate_share_account_payload_v1(user_id = 1,access_level= ShareAccount_V1_AccessLevel.CAN_VIEW))
print(generate_share_account_payload_v2(user_id =1, access_level= ShareAccount_V2_AccessLevel.CAN_VIEW))


{'type': 'USER', 'id': 1, 'permissions': ['READ']}
{'type': 'USER', 'id': 1, 'accessLevel': 'CAN_VIEW'}


In [15]:
#| export
async def share_account_v2(auth: dmda.DomoAuth,
                           account_id: str,
                           share_payload: dict,
                           debug_api: bool = False,
                           session: httpx.AsyncClient = None
                           ):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v2/accounts/share/{account_id}"

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=share_payload,
        debug_api=debug_api,
        session=session
    )

#v1 may have been deprecated.  used to be tied to group beta
async def share_account_v1(auth: dmda.DomoAuth,
                           account_id: str,
                           share_payload: dict,
                           debug_api: bool = False,
                           session: httpx.AsyncClient = None
                           ):

    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/accounts/{account_id}/share"

    return await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        body=share_payload,
        debug_api=debug_api,
        session=session
    )


In [16]:
# | hide
import nbdev

nbdev.nbdev_export()
